<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_2020/blob/master/ANN_fotos_testa_variaveis_02_jan_20_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import math
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [0]:
#@title Digite os parametros { run: "auto", vertical-output: true }
#@markdown Enter neurons hidden layer
N_Hidden=18  #@param {type:"number"}
#@markdown Activation function

Ativa='logistic'#@param ["tanh", "identity", "logistic","relu"] {allow-input: true}

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [5]:
!wget http://ftp.feq.ufu.br/ubirajara/Fotos_Treina_ANN_dez_2019.csv
Transfere='Fotos_Treina_ANN_dez_2019.csv'

--2020-01-20 15:05:27--  http://ftp.feq.ufu.br/ubirajara/Fotos_Treina_ANN_dez_2019.csv
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21508 (21K) [text/csv]
Saving to: ‘Fotos_Treina_ANN_dez_2019.csv.4’

Fotos_Treina_ANN_de 100%[===================>]  21.00K  22.1KB/s    in 1.0s    

2020-01-20 15:05:29 (22.1 KB/s) - ‘Fotos_Treina_ANN_dez_2019.csv.4’ saved [21508/21508]



In [0]:
df= pd.read_csv(Transfere) 

In [7]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
print(df.columns)

Index(['Unnamed: 0', 'Type', 'no.peaks', 'Media_proem', 'area under',
       'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode',
       'Mean', 'Sd'],
      dtype='object')


In [0]:
df['Type'] = df['Type'].map({'buraco':1, 'grao':2, 'indef':3}) 

In [9]:

Colunms=['no.peaks', 'Media_proem', 'area under','Width_peaks',
         'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode',
         'Mean', 'Sd']
'''
Colunms=['no.peaks', 'Media_proem', 'area under','Width_peaks',
         'Median', 'Mode',
         'Mean', 'Sd']
'''
#X=df.iloc[:,2:12]
#y=df.iloc[:,1].astype(int)
X=df[Colunms]
#y=df['Type'].astype(int)
y=df['Type']
print(y)

0      1
1      1
2      1
3      1
4      1
      ..
115    3
116    3
117    3
118    3
119    3
Name: Type, Length: 120, dtype: int64


In [10]:
print(df.corr(method ='pearson'))

                 Unnamed: 0      Type  no.peaks  Media_proem  area under  Width_peaks  Width_peaks_max  Width_peaks_min    Median      Mode      Mean        Sd
Unnamed: 0       1.000000    0.000000  0.112320 -0.102386    -0.076443   -0.074573     0.014809        -0.102254        -0.146656 -0.026923 -0.075613  0.033739
Type             0.000000    1.000000 -0.139996  0.216028    -0.154951    0.125141     0.020333        -0.026574        -0.040544  0.052604 -0.160519 -0.309648
no.peaks         0.112320   -0.139996  1.000000 -0.162566    -0.141823   -0.221004     0.330808        -0.371817        -0.160602 -0.085007 -0.141183 -0.033249
Media_proem     -0.102386    0.216028 -0.162566  1.000000     0.113582    0.794297     0.466941         0.171858         0.149918  0.070092  0.111897  0.028565
area under      -0.076443   -0.154951 -0.141823  0.113582     1.000000    0.326182     0.080327         0.235838         0.870330  0.720729  0.999956  0.813025
Width_peaks     -0.074573    0.125141 -0

In [21]:
df2=df.corr(method ='pearson')
print(df2)

                 Unnamed: 0      Type  no.peaks  Media_proem  area under  Width_peaks  Width_peaks_max  Width_peaks_min    Median      Mode      Mean        Sd
Unnamed: 0       1.000000    0.000000  0.112320 -0.102386    -0.076443   -0.074573     0.014809        -0.102254        -0.146656 -0.026923 -0.075613  0.033739
Type             0.000000    1.000000 -0.139996  0.216028    -0.154951    0.125141     0.020333        -0.026574        -0.040544  0.052604 -0.160519 -0.309648
no.peaks         0.112320   -0.139996  1.000000 -0.162566    -0.141823   -0.221004     0.330808        -0.371817        -0.160602 -0.085007 -0.141183 -0.033249
Media_proem     -0.102386    0.216028 -0.162566  1.000000     0.113582    0.794297     0.466941         0.171858         0.149918  0.070092  0.111897  0.028565
area under      -0.076443   -0.154951 -0.141823  0.113582     1.000000    0.326182     0.080327         0.235838         0.870330  0.720729  0.999956  0.813025
Width_peaks     -0.074573    0.125141 -0

In [11]:
print(df.head())

   Unnamed: 0  Type  no.peaks  Media_proem    area under  Width_peaks  Width_peaks_max  Width_peaks_min      Median      Mode       Mean         Sd
0  0           1     4.0       51.384595    2993.276979   38.492219    100.229477       0.616279         4.664086    1.576266  18.934430  21.464729
1  1           1     2.0       47.626771    1315.847022   28.334735    60.077800        1.577800         3.425990    1.186329  8.392858   10.178723
2  2           1     2.0       47.249986    1709.811993   35.353698    104.468318       1.018112         5.813794    1.019165  10.898776  14.800251
3  3           1     4.0       70.047513    12391.952111  87.710404    156.975006       1.441023         102.551759  6.722350  77.750603  45.762334
4  4           1     6.0       42.730606    1288.159588   30.966763    132.500000       0.508409         5.192788    1.336058  8.154355   8.389805 


In [12]:

# separa os dados variaveis de entrada em normatizados e originais
X_OLD=[]
X_OLD=X.copy()
X=Normatiza(X)
#print(X)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=25)

In [0]:
# define a rede neural 
#N_Hidden=(18,18,18)


clf = MLPClassifier(solver='lbfgs',activation=Ativa,alpha=1e-5,
                    hidden_layer_sizes=N_Hidden,random_state=0)


In [15]:
# treina a rede neural
clf.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=18, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=clf.predict(X_train)
y_pred=clf.predict(X_test)


In [17]:
# Evaluating the Algorithm
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[5 4 2]
 [2 1 1]
 [1 3 5]]
              precision    recall  f1-score   support

           1       0.62      0.45      0.53        11
           2       0.12      0.25      0.17         4
           3       0.62      0.56      0.59         9

    accuracy                           0.46        24
   macro avg       0.46      0.42      0.43        24
weighted avg       0.54      0.46      0.49        24



In [18]:
'''
[[5 4 2]
 [2 1 1]
 [1 3 5]]
              precision    recall  f1-score   support

           1       0.62      0.45      0.53        11
           2       0.12      0.25      0.17         4
           3       0.62      0.56      0.59         9

    accuracy                           0.46        24
   macro avg       0.46      0.42      0.43        24
weighted avg       0.54      0.46      0.49        24
'''

'\n[[5 4 2]\n [2 1 1]\n [1 3 5]]\n              precision    recall  f1-score   support\n\n           1       0.62      0.45      0.53        11\n           2       0.12      0.25      0.17         4\n           3       0.62      0.56      0.59         9\n\n    accuracy                           0.46        24\n   macro avg       0.46      0.42      0.43        24\nweighted avg       0.54      0.46      0.49        24\n'

In [19]:
print(y_pred)

[2 1 1 1 1 1 3 2 3 1 3 1 1 1 2 3 3 3 3 3 3 1 1 2]
